<a href="https://colab.research.google.com/github/elisasmenendez/inforank/blob/master/pagerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank with SPARQL Queries

In the RDF world, we typically store graphs in triplestores with SPARQL endpoints. Hence, in this implementation, we show how to compute the classic PageRank score using SPARQL queries.

# Running Example

To exemplify the implementation steps, consider the simple graph shown in the following image with IRIs denoted in oval and literals denoted in dashed boxes (Colab image is not working).

https://drive.google.com/uc?export=view&id=1NHAl72vWPRlFOiQet9bF5--VjxsQStIZ

Consider that the graph also includes the following triples: 
 (X, rdf:type, rdfs:Class), 
 (Y, rdf:type, rdfs:Class), 
 (Z, rdf:type, rdfs:Class)
 (x1, rdf:type, X), 
 (x2, rdf:type, X), 
 (y1, rdf:type, Y), 
 (y2, rdf:type, Y),
 (z1, rdf:type, Z), 
 (z2, rdf:type, Z), … (z10, rdf:type, Z)

The example graph is stored as a n-triple file. Let's see how to load it with RDFLib.


In [ ]:
pip install rdflib

In [2]:
import rdflib

g = rdflib.Graph()
g.parse("https://raw.githubusercontent.com/elisasmenendez/inforank/master/example.ttl", format="turtle")

g.bind("ex", "http://example.com/")
g.bind("quira", "http://quira.com/")

And how to print all triples.

In [ ]:
qres = g.query("select ?s ?p ?o where {?s ?p ?o }",)

for row in qres:
  print ("%s %s %s" % row)

# PageRank - The classic approach

To achieve the classic PageRank score, we first need to compute the degree of the each instance.

In [3]:
# Computing the degree of instances
g.update("""
insert { ?r quira:degree ?degree }
where { 
  select ?r ((?out + ?in) as ?degree)
  where {
    {
      select ?r (count(?o) as ?out)
      where { 
        ?r rdf:type/rdf:type rdfs:Class .
        OPTIONAL { 
          ?r ?p ?o . 
          ?o rdf:type/rdf:type rdfs:Class 
        }
      }
      group by ?r 
    }
    {
      select ?r (count(?o) as ?in)
      where { 
        ?r rdf:type/rdf:type rdfs:Class .
        OPTIONAL { 
          ?o ?p ?r . 
          ?o rdf:type/rdf:type rdfs:Class 
        }
      }
      group by ?r
    }
  }
}
""")

Now, we initialize the PageRank scores with *1/n*, in which *n* is the number of instances in the graph.

In [7]:
# Counting the number of instances in the graph
qres = g.query("""
select (count(*) as ?n)
where { ?r rdf:type/rdf:type rdfs:Class }
""")
n = qres.bindings[0]['n']
last = 'score1'
curr = 'score2'

# Initializing all scores with 1/N
g.update("""
insert { ?r quira:%s ?score }
where { 
  ?r rdf:type/rdf:type rdfs:Class .
  BIND( (1/%s) as ?score )
}
""" % (last,n) )

Finally, we execute the Power Iteration Method to compute PageRank.

In [ ]:
# This query simulates an iteration with a dumping factor of 0.85  
queryIter = """
insert { ?r quira:%s ?score }
where {
  select ?r ((((1-0.85)/%s) + (0.85 * sum(?score/?degree))) as ?score)
  where { 
    { select *
      where {
        ?r rdf:type/rdf:type rdfs:Class .
        ?s ?p ?r .
        ?s quira:%s ?score .
        ?s quira:degree ?degree .
      }
    }
    UNION
    { select *
      where {
        ?r rdf:type/rdf:type rdfs:Class .
        ?r ?p ?s .
        ?s quira:%s ?score .
        ?s quira:degree ?degree .
      }
    } 
  }
  group by ?r
}
"""

# This query simulates the convergence calculation 
queryConv = """
select (sum(abs(?curr - ?last)) as ?conv)
where { 
  ?r quira:%s ?curr .
  ?r quira:%s ?last . 
}
"""

# Executing the Power Iteration method
tol = 1.0e-4
max = 100
converged = 0

for i in range(max-1):
  
  # Iterating
  g.update(queryIter % (curr, n, last, last))
  
  # Checking convergence
  qres = g.query(queryConv % (curr, last))
  conv = float(qres.bindings[0]['conv'])

  #print("Iteration %s - Convergence %s" % (i, conv))  

  if conv < tol:
    converged = 1
    g.update("insert { ?r quira:pagerank ?score } where { ?r quira:%s ?score }" % curr)
    g.update("delete where { ?s quira:%s ?o }" % last)
    g.update("delete where { ?s quira:%s ?o }" % curr)
    print("Converged after %s iterations" % i) 
    break
  else:
    g.update("delete where { ?s quira:%s ?o }" % last)
    temp = last
    last = curr
    curr = temp

if converged == 0:
  print("Failed to converge after %s iterations" % i) 

Check the results

In [ ]:
qres = g.query("""
select ?r ?score
where {?r quira:pagerank ?score }  
order by desc(?score)
""")

for row in qres:
  print ("%s %s" % row)

Compare with the result given by Networkx

In [ ]:
import networkx as nx
import operator

edges = [
  ('X1', 'Y1'),
  ('X2', 'Y1'),
  ('Y2', 'Y1'),
  ('Z1', 'Y2'),
  ('Z2', 'Y2'),
  ('Z3', 'Y2'),
  ('Z4', 'Y2'),
  ('Z5', 'Y2'),
  ('Z6', 'Y2'),
  ('Z7', 'Y2'),
  ('Z8', 'Y2'),
  ('Z9', 'Y2'),
  ('Z10', 'Y2')]

G = nx.Graph()  
G.add_edges_from(edges)  

res = nx.pagerank(G)

sorted_res = sorted(res.items(), key=operator.itemgetter(1), reverse=True)  
for i,j in sorted_res:
  print(i + " - " + str(j))
